https://medium.com/daangn/pytorch-multi-gpu-%ED%95%99%EC%8A%B5-%EC%A0%9C%EB%8C%80%EB%A1%9C-%ED%95%98%EA%B8%B0-27270617936b

## 딥러닝과 GPUs

딥러닝은 Deep Neural Netowrk (DNN)라는 arhitecture에 따르는 매트릭스 연산을 수행하기 때문에 GPU를 사용할 경우 연산처리 속도가 상당히 빨라집니다. 특히, 이미지 처리를 하는 경우에는 더욱이 큰 장점을 가지고 있다고 알려져 있습니다. 그리고 비전 분야에서는 DNN을 깊게 쌓는 방식으로 ImageNet Classification의 정확도가 상당히 높아짐에 따라 backbone이라는 특정 DNN architecture가 많이 나오고 있습니다. Resnet도 이중의 하나로서 그 출발점이라고도 할 수 있고, 이 이후로는 상항히 큰 데이터셋에 대해서 깊은 모델을 사용함으로서 성능을 발전시키고 있습니다. 

<img src='./imgs/rev.png'>

데이터 셋과 모델이 모두 커짐에 따라 연산량이 또한 매우 많기 때문에 이러한 연산처리를 해결하기 위해서 여러 개의 GPU를 사용하는 것이 매우 효율적입니다. 그리고 이 때 batchsize가 성능에 영향을 주는 경우가 많은데 이는 여러 GPU의 메모리 사용 불균형과 연관이 있으며 반드시 해결해야합니다. 

> 데이터 셋과 모델이 매우 크기 때문에 단순히 batchsize의 차이로 학습속도가 작게는 몇 시간에서 1, 2주일까지 차이가 날 수도 있습니다.

multi-GPU는 단순히 4개의 GPU를 한 대의 컴퓨터에 설치한 것이기도 하지만, 이를 효율적으로 사용하는 것이 가장 중요합니다. 예를 들어, 메모리의 사용량, 딥러닝의 framework에서의 multi-processing 설계, batchsize 등을 고려해야합니다. 

## Pytorch와 multi-gpu

### multi-gpu를 통한 학습과정
pytorch에서는 multi-gpu의 사용에 대한 편의성을 제공하기 위한 여러 가지 기능을 제공하고 있으며, `Data Parallel`도 그 중 하나입니다. 다음의 reference를 참고하여 작성하였습니다.

- reference: [Training Neural Nets on Larger Batches: Practical Tips for 1-GPU, Multi-GPU & Distributed setups](https://medium.com/huggingface/training-larger-batches-practical-tips-on-1-gpu-multi-gpu-distributed-setups-ec88c3e51255)

다음의 그림은 `Data Paralle`이 어떻게 작동하는지에 대한 여러 가지 방식을 보여줍니다.
<img src='./imgs/data_parallel.png'>

앞서 말했듯이, pytorch는 multi-gpu를 사용하기 위한 여러 기능(함수)들을 제공하고 있습니다. 그렇기 때문에 새롭게 코딩을 하기 보다는 제공하는 함수를 사용하여서 다음과 같이 multi-gpu를 사용할 수 있습니다. 먼저, GPU가 여러 개일 경우에 대한 딥러닝의 학습은 다음과 같은 과정을 거칩니다. 일단, 여러 개의 GPU로 하나의 모델을 학습시키기 위해서는 각 GPU에 모델을 복사해서 할당해야하고 이는 `Data Parallel`에서 제공하는 `replicate`를 통해서 이루어집니다. 그리고 매 iteration을 수행할 때마다 batch를 GPU의 개수만큼 나누며, 이러한 과정을 `scatter`라고 합니다. 이렇게 입력을 나누면 각 GPU에서 forward 과정을 통해서 각 입력에 대해 모델이 출력을 생성하고 하나의 GPU로 모으며, 이렇게 tensor를 하나의 device로 모으는 것을 `gather` 함수가 수행합니다.

forward 과정을 수행했다면, loss function을 통해서 loss를 계산하고 back-propagation을 진행합니다. back-propagation은 각 GPU에서 수행하고 그 결과로 각 GPU에 복사된 모델들의 gradient를 구할 수 있습니다. 그리고 모델을 업데이트하기 위해서는 각 GPU에서 구한 gradient들을 또 하나의 GPU로 모아서 업데이트를 합니다. 

> 선택한 optimizer에 따라서 업데이트를 바로 하지 않고 추가 연산을 할 수도 있습니다. 예를 들어서, Adam은 gradient로 바로 모델을 업데이트하지 않고 추가 연산을 수행합니다.

요약하자면, `replicate` $\rightarrow$ `scatter` $\rightarrow$ `gather` $\rightarrow$ `gather`의 과정을 하는 것이고, 모두 pytorch의 `Data Parallel`에 구축되어 있기 때문에 다음과 같이 한줄로서 수행이 가능합니다.  

```
model = nn.DataParallel(model)
```

이에 대한 내부 코드는 다음과 같습니다. 

```

def data_parallel(module, input, device_ids, output_device):
    replicas = nn.parallel.replicate(module, device_ids)
    inputs = nn.parallel.scatter(input, device_ids)
    replicas = replicas[:len(inputs)]
    outputs = nn.parallel.parallel_apply(replicas, inputs)
    return nn.parallel.gather(outputs, output_device)
```

### 메모리의 불균형 문제해결

앞서 말했듯이, 메모리의 불균형 문제는 학습속도에 많은 영향을 끼치기 때문에 반드시 해결해야합니다. 특히, 여러 개의 GPU로부터의 출력이 모아지는 GPU는 혼자서만 loss function함수를 수행하기 때문에 메모리의 사용량이 클 수밖에 없습니다. 그렇기 때문에 `model = nn.DataParallel(model)`라고 하면, 디폴트로 정해져있는 GPU로 출력이 모아지게 됩니다. 그리고 이를 임의로 설정하여 다른 GPU로 출력이 모아지도록 할 수는 있지만, 이 또한 마찬가지로 하나의 GPU에 대해서만 메모리 사용량이 많아질 것이기 때문에 해결방법이 되지 못합니다. 예를 들어, 3개의 GPU를 사용한다면 다음과 같이 가능합니다. 

```
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2'
model = nn.DataParallel(mode, output_device=1)
```

그리고 1번의 GPU에 대해서만 다시 메모리 사용량이 많아질 것입니다. 결국에는 출력이 모아지는 GPU에서만 수행하는 loss function도 마찬가지로 각각의 GPU로 분배를 해야 메모리의 사용량을 공평하게 분해할 수 있게 됩니다.

앞서, 모델을 각각의 GPU에 복사한 것과 마찬가지로 loss function 또한 pytorch에서는 하나의 모듈로서 제공되기 때문에 각 GPU에 복사할 수 있습니다. 그리고 loss function을 계산하기 위한 출력의 `target` tensor도 전달하여 각각의 GPU가 모두 독립적으로 loss function을 수행할 수 있고, 계산한 `loss`를 통해서 backward도 가능합니다. 이러한 과정을 하나의 모듈로서 pytorch의 `DataParallel`처럼 제공하는 코드가 **[Pytorch-Encoding](https://github.com/zhanghang1989/PyTorch-Encoding)**으로 존재합니다.

<img src='./imgs/f_l_b.png'>

이러한 과정은 다음의 내부코드를 통해서 살펴볼 수 있습니다. 

```

from torch.nn.parallel.data_parallel import DataParallel

class DataParallelCriterion(DataParallel):
    def forward(self, inputs, *targets, **kwargs):
        targets, kwargs = self.scatter(targets, kwargs, self.device_ids)
        replicas = self.replicate(self.module, self.device_ids[:len(inputs)])
        targets = tuple(targets_per_gpu[0] for targets_per_gpu in targets)
        outputs = _criterion_parallel_apply(replicas, inputs, targets, kwargs)
        return Reduce.apply(*outputs) / len(outputs), targets
```

위에서처럼 pytorch는 `DataParallelCriterion`이라는 모듈을 제공하고, `target`을 `scatter`하고, `replcate`한 모듈에서 각각 계산을 수행합니다. 그리고 계산한 `outputs`와 `Reduce.apply`를 통해서 각각의 GPU에서 backward를 수행하도록 합니다. 이 모듈을 사용하기 위해서는 다음과 같이 기존의 `DataParallel`이 아닌 `DataParallelModel`을 사용해야 합니다. 간단한 사용 방법으로는 Pytorch-Encoding에서 **[paralle.py](https://github.com/zhanghang1989/PyTorch-Encoding/blob/master/encoding/parallel.py)**만 복사하여 import하여 다음과 같이 사용가능합니다.

```
import torch
import torch.nn as nn
from parallel import DataParallelModel, DataParallelCriterion

model = BERT(args)
model = DataParallelModel(model)
model.cuda()

criterion = nn.NLLLoss()
criterion = DataParallelCriterion(criterion) 
```


### GPU-Util 100% 사용하기

* reference
    - [pytorch 분산학습](https://pytorch.org/tutorials/intermediate/dist_tuto.html)